In [1]:
data = """
# Virat Kohli - Centuries

Virat Kohli is one of the most successful and consistent batsmen in modern-day cricket. Below is a table summarizing his centuries in international cricket (as of March 2025).

## List of Virat Kohli's Centuries

| **No.** | **Format**    | **Opponent**    | **Match Date** | **Venue**           | **Score** |
|---------|---------------|-----------------|----------------|---------------------|-----------|
| 1       | ODI           | Sri Lanka       | 2013-07-21     | Colombo             | 102*      |
| 2       | Test          | Australia       | 2014-12-17     | Adelaide            | 115       |
| 3       | ODI           | West Indies     | 2014-10-11     | North Sound         | 100       |
| 4       | ODI           | Sri Lanka       | 2014-11-01     | Ranchi              | 139       |
| 5       | ODI           | South Africa    | 2015-10-14     | Kanpur              | 138       |
| 6       | Test          | Sri Lanka       | 2017-07-29     | Galle               | 103       |
| 7       | ODI           | New Zealand     | 2017-10-22     | Pune                | 121       |
| 8       | Test          | Bangladesh      | 2017-02-09     | Hyderabad           | 204       |
| 9       | ODI           | Sri Lanka       | 2017-08-21     | Colombo             | 103*      |
| 10      | ODI           | Australia       | 2019-03-10     | Ranchi              | 116       |
| 11      | ODI           | West Indies     | 2019-08-14     | Port of Spain       | 114       |
| 12      | Test          | Australia       | 2018-12-06     | Adelaide            | 123       |
| 13      | T20I          | Sri Lanka       | 2016-03-22     | Mohali              | 82*       |
| 14      | Test          | West Indies     | 2019-08-30     | Kingston            | 114       |
| 15      | ODI           | South Africa    | 2018-02-14     | Johannesburg        | 160       |

This table highlights some of the notable centuries scored by Virat Kohli. He has been an exceptional performer across all formats, cementing his place as one of cricket's all-time greats.
"""

In [2]:
!pip install -q chonkie tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00


In [3]:
from chonkie import RecursiveChunker, RecursiveLevel, RecursiveRules

In [4]:
rules = RecursiveRules(
    levels=[
        RecursiveLevel(delimiters=["######", "#####", "####", "###", "##", "#"], include_delim="next"),
        RecursiveLevel(delimiters=["\n\n", "\n", "\r\n", "\r"]),
        RecursiveLevel(delimiters=["\n|"], include_delim="next"),
        RecursiveLevel(delimiters=".?!;:"),
        RecursiveLevel(),
    ]
)

In [5]:
chunker = RecursiveChunker(rules=rules, chunk_size=384, return_type="texts")
# chunker = RecursiveChunker(rules=rules, chunk_size=384)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
# This is all it takes to chunk!
chunks = chunker(data)
print(f"Total number of chunks: {len(chunks)}")

Total number of chunks: 4


In [7]:
# print(chunks[0].text)
print(chunks[0])


# Virat Kohli - Centuries

Virat Kohli is one of the most successful and consistent batsmen in modern-day cricket. Below is a table summarizing his centuries in international cricket (as of March 2025).




In [8]:
print(chunks[1])

## List of Virat Kohli's Centuries

| **No.** | **Format**    | **Opponent**    | **Match Date** | **Venue**           | **Score** |
|---------|---------------|-----------------|----------------|---------------------|-----------|
| 1       | ODI           | Sri Lanka       | 2013-07-21     | Colombo             | 102*      |
| 2       | Test          | Australia       | 2014-12-17     | Adelaide            | 115       |
| 3       | ODI           | West Indies     | 2014-10-11     | North Sound         | 100       |
| 4       | ODI           | Sri Lanka       | 2014-11-01     | Ranchi              | 139       |



In [9]:
print(chunks[2])

| 5       | ODI           | South Africa    | 2015-10-14     | Kanpur              | 138       |
| 6       | Test          | Sri Lanka       | 2017-07-29     | Galle               | 103       |
| 7       | ODI           | New Zealand     | 2017-10-22     | Pune                | 121       |
| 8       | Test          | Bangladesh      | 2017-02-09     | Hyderabad           | 204       |
| 9       | ODI           | Sri Lanka       | 2017-08-21     | Colombo             | 103*      |
| 10      | ODI           | Australia       | 2019-03-10     | Ranchi              | 116       |



In [10]:
def process_chunks(chunks):
    """
    Process markdown chunks to ensure table parts maintain their context.

    If a chunk contains the beginning of a table (headers), this function will
    identify the table title and headers. It will then prepend these to any
    subsequent chunks that continue the same table but lack headers.

    Args:
        chunks (list): List of string chunks from a markdown file

    Returns:
        list: Processed chunks with table headers added where needed
    """
    processed_chunks = []
    current_table_title = None
    current_table_headers = None
    table_active = False

    for i, chunk in enumerate(chunks):
        # Check if this chunk contains a table
        if "|" in chunk and "|-" in chunk:
            # This chunk contains a table with headers

            # Extract table title (could be a header or a sentence above the table)
            lines = chunk.split('\n')
            title_line = None

            # Look for title: either a heading or the last text line before the table
            for j, line in enumerate(lines):
                if line.strip() and "|" in line:
                    # Found the first line of the table
                    # The title should be the last non-empty line before this
                    for k in range(j-1, -1, -1):
                        if lines[k].strip() and "|" not in lines[k]:
                            title_line = lines[k].strip()
                            break
                    break

            # If no title found but there are headers (h1, h2, etc.), use the last one
            if not title_line:
                for line in lines:
                    if line.strip().startswith('#'):
                        title_line = line.strip()

            # Find the header row and separator row
            header_row = None
            separator_row = None

            for j in range(len(lines)):
                if "|" in lines[j] and header_row is None:
                    # First row with | is likely the header
                    header_row = lines[j]

                    # Next row with | and - is likely the separator
                    if j+1 < len(lines) and "|" in lines[j+1] and "-" in lines[j+1]:
                        separator_row = lines[j+1]
                        break

            if header_row and separator_row:
                current_table_headers = header_row + '\n' + separator_row
                current_table_title = title_line
                table_active = True
                processed_chunks.append(chunk)
            else:
                # This chunk has a table-like format but no proper headers
                processed_chunks.append(chunk)

        elif "|" in chunk and table_active:
            # This chunk continues a table without headers

            # Create a new chunk with the title and headers prepended
            enhanced_chunk = ""

            # Add the title if available
            if current_table_title:
                enhanced_chunk += current_table_title + "\n\n"

            # Add the headers
            if current_table_headers:
                enhanced_chunk += current_table_headers + "\n"

            # Add the original chunk content
            enhanced_chunk += chunk

            processed_chunks.append(enhanced_chunk)
        else:
            # This is not a table chunk, or it's a new section
            table_active = False
            current_table_headers = None
            current_table_title = None
            processed_chunks.append(chunk)

    return processed_chunks

In [11]:
processed_chunks = process_chunks(chunks)

print("PROCESSED CHUNKS:")
for i, chunk in enumerate(processed_chunks):
    print(f"\nCHUNK {i+1}:")
    print(chunk)

PROCESSED CHUNKS:

CHUNK 1:

# Virat Kohli - Centuries

Virat Kohli is one of the most successful and consistent batsmen in modern-day cricket. Below is a table summarizing his centuries in international cricket (as of March 2025).



CHUNK 2:
## List of Virat Kohli's Centuries

| **No.** | **Format**    | **Opponent**    | **Match Date** | **Venue**           | **Score** |
|---------|---------------|-----------------|----------------|---------------------|-----------|
| 1       | ODI           | Sri Lanka       | 2013-07-21     | Colombo             | 102*      |
| 2       | Test          | Australia       | 2014-12-17     | Adelaide            | 115       |
| 3       | ODI           | West Indies     | 2014-10-11     | North Sound         | 100       |
| 4       | ODI           | Sri Lanka       | 2014-11-01     | Ranchi              | 139       |


CHUNK 3:
## List of Virat Kohli's Centuries

| **No.** | **Format**    | **Opponent**    | **Match Date** | **Venue**           | **Scor